In [5]:
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches

### Train Pairs Generator ###

In [42]:
orig_file='/nfs2/baos1/rudravg/GCA062ACA_TISSUE01_DAPI_DAPI_12ms_ROUND_00_initial_reg.tif'
mask_file='/nfs2/baos1/rudravg/Retention_Masks/GCA062ACA_TISSUE01_TISSUE_RETENTION.tif'
file_1_name='/nfs2/baos1/rudravg/GCA062ACA_TISSUE01_DAPI_DAPI_12ms_ROUND_16_initial_reg.tif'
file_2_name='/nfs2/baos1/rudravg/GCA062ACA_TISSUE01_DAPI_DAPI_12ms_ROUND_14_initial_reg.tif'
file_3_name='/nfs2/baos1/rudravg/GCA062ACA_TISSUE01_DAPI_DAPI_12ms_ROUND_17_initial_reg.tif'

parts = orig_file.split('/')[-1].split('_')
subject_name = parts[0]
tissue_numer=parts[1]
orig_round_number = parts[-3]
parts=file_1_name.split('/')[-1].split('_')
file_1_round_number=parts[-3]
parts=file_2_name.split('/')[-1].split('_')
file_2_round_number=parts[-3]
parts=file_3_name.split('/')[-1].split('_')
file_3_round_number=parts[-3]
#Print all round numbers
print('Subject Name:',subject_name)
print('Original Round Number:',orig_round_number)
print('File 1 Round Number:',file_1_round_number)
print('File 2 Round Number:',file_2_round_number)
print('File 3 Round Number:',file_3_round_number)



mask_file=np.array(Image.open(mask_file))
#Make the mask file binary
mask_file[mask_file>0]=1
orig_file=np.array(Image.open(orig_file))
orig_file=(orig_file*mask_file)/255.0
file_1=np.array(Image.open(file_1_name))
file_1=(file_1*mask_file)/255.0
file_2=np.array(Image.open(file_2_name))
file_2=(file_2*mask_file)/255.0
file_3=np.array(Image.open(file_3_name))
file_3=(file_3*mask_file)/255.0
orig_file.dtype



Subject Name: GCA062ACA
Original Round Number: 00
File 1 Round Number: 16
File 2 Round Number: 14
File 3 Round Number: 17


dtype('float64')

In [43]:
window_size = (1024, 1024)

# Define directories
output_dir = '/home-local/rudravg/test_DAPI'
target_dir = os.path.join(output_dir, 'target_images')
source_dir = os.path.join(output_dir, 'source_images')

# Create directories if they don't exist
os.makedirs(target_dir, exist_ok=True)
os.makedirs(source_dir, exist_ok=True)

# Define pairs file
pairs_file = os.path.join(output_dir, 'pairs.txt')

# Open pairs file for writing
with open(pairs_file, 'a') as f:
    for i in range(0, mask_file.shape[0], window_size[0]):
        for j in range(0, mask_file.shape[1], window_size[1]):
            window_mask = mask_file[i:i+window_size[0], j:j+window_size[1]]
            
            if np.count_nonzero(window_mask) > 200:
                # Save the cut pieces of the orig_file in target_dir
                window_orig = orig_file[i:i+window_size[0], j:j+window_size[1]]
                window_orig = (window_orig * 255).astype(np.uint8)
                target_path = os.path.join(target_dir, f'{subject_name}_{tissue_numer}_{orig_round_number}_{i}_{j}.npy')
                np.save(target_path, window_orig)
                
                # Cut file_1, file_2 and file_3 in the same way and save their corresponding pieces in source_dir
                for file, file_name, round_number in zip([file_1, file_2, file_3], ['file_1', 'file_2', 'file_3'], [file_1_round_number, file_2_round_number, file_3_round_number]):
                    window_file = file[i:i+window_size[0], j:j+window_size[1]]
                    window_file = (window_file * 255).astype(np.uint8)
                    source_path = os.path.join(source_dir, f'{subject_name}_{tissue_numer}_{round_number}_{i}_{j}.npy')
                    np.save(source_path, window_file)
                    
                    # Append a space separated line in pairs_file with the first part being path to the source image and the second part being path to the target image
                    f.write(f'{source_path} {target_path}\n')

### Val Pairs Generator ###

In [50]:
orig_file='/nfs2/baos1/rudravg/GCA045TIB_TISSUE01_DAPI_DAPI_12ms_ROUND_00_initial_reg.tif'
mask_file='/nfs2/baos1/rudravg/Retention_Masks/GCA045TIB_TISSUE01_TISSUE_RETENTION.tif'
file_1_name='/nfs2/baos1/rudravg/GCA045TIB_TISSUE01_DAPI_DAPI_12ms_ROUND_13_initial_reg.tif'

parts = orig_file.split('/')[-1].split('_')
subject_name = parts[0]
tissue_numer=parts[1]
orig_round_number = parts[-3]
parts=file_1_name.split('/')[-1].split('_')
file_1_round_number=parts[-3]
#Print all round numbers
print('Subject Name:',subject_name)
print('Original Round Number:',orig_round_number)
print('File 1 Round Number:',file_1_round_number)

mask_file=np.array(Image.open(mask_file))
#Make the mask file binary
mask_file[mask_file>0]=1
orig_file=np.array(Image.open(orig_file))
orig_file=(orig_file*mask_file)/255.0
file_1=np.array(Image.open(file_1_name))
file_1=(file_1*mask_file)/255.0
orig_file.dtype

Subject Name: GCA045TIB
Original Round Number: 00
File 1 Round Number: 13


dtype('float64')

In [51]:
window_size = (512, 512)

# Define directories
output_dir = '/home-local/rudravg/test_DAPI'
target_dir = os.path.join(output_dir, 'val_target_images')
source_dir = os.path.join(output_dir, 'val_source_images')

# Create directories if they don't exist
os.makedirs(target_dir, exist_ok=True)
os.makedirs(source_dir, exist_ok=True)

# Define pairs file
pairs_file = os.path.join(output_dir, 'val_pairs.txt')

# Open pairs file for appending
with open(pairs_file, 'a') as f:
    for i in range(0, mask_file.shape[0], window_size[0]):
        for j in range(0, mask_file.shape[1], window_size[1]):
            window_mask = mask_file[i:i+window_size[0], j:j+window_size[1]]
            
            if np.count_nonzero(window_mask) > 200:
                # Save the cut pieces of the orig_file in target_dir
                window_orig = orig_file[i:i+window_size[0], j:j+window_size[1]]
                window_orig = (window_orig * 255).astype(np.uint8)
                target_path = os.path.join(target_dir, f'{subject_name}_{orig_round_number}_{i}_{j}.npy')
                np.save(target_path, window_orig)
                
                # Cut file_1 and save its corresponding piece in source_dir
                window_file_1 = file_1[i:i+window_size[0], j:j+window_size[1]]
                window_file_1 = (window_file_1 * 255).astype(np.uint8)
                source_path = os.path.join(source_dir, f'{subject_name}_{file_1_round_number}_{i}_{j}.npy')
                np.save(source_path, window_file_1)
                    
                # Append a space separated line in pairs_file with the first part being path to the source image and the second part being path to the target image
                f.write(f'{source_path} {target_path}\n')

### To delete all the non 512x512 images pairs from the txt file ###
Note that the images are still saved in the directories, just the difference is that the new_pairs.txt and new_val_pairs.txt won't load them

In [6]:
import os
import numpy as np
from tqdm import tqdm

# Define the directory
dir_path = '/home-local/rudravg/test_DAPI/val_source_images'
non_512_filenames = []
# Iterate over all files in the directory
for filename in tqdm(os.listdir(dir_path)):
    # Check if the file is a .npy file
    if filename.endswith('.npy'):
        # Load the numpy array
        array = np.load(os.path.join(dir_path, filename))
        
        # Check if the shape of the array is not 512x512
        if array.shape != (512, 512):
            non_512_filenames.append(filename)
print(len(non_512_filenames))
new_txt_file = '/home-local/rudravg/test_DAPI/new_val_pairs.txt'
txt_file = '/home-local/rudravg/test_DAPI/val_pairs.txt'

# Open the text file for reading and the new text file for writing
with open(txt_file, 'r') as f, open(new_txt_file, 'w') as new_f:
    # Iterate over all lines in the text file
    for line in f:
        # Check if the line contains any of the non-512x512 filenames
        if not any(filename in line for filename in non_512_filenames):
            # Write the line to the new text file
            new_f.write(line)

100%|██████████| 108/108 [00:00<00:00, 8861.03it/s]

6
